In [ ]:
# default_exp derivative

# stats

> tools for statistical analysis

In [ ]:
#export
import xarray as xr
import numpy as np

In [ ]:
#export
def regression(data, index):
    """
    description:
        Fast regression for spatial data.
    input:
        data (nd array) - multidimensional array with time as first dimension
        index (1d array) - needs to have the same length as time dim of data
    returns:
        b0, b1 (nd array) - b0 + b1 * x
    """

    data_flat = data.reshape(data.shape[0], np.product(data.shape[1:]))
    
    index = np.vstack((np.ones(len(index)), index)).T
    beta_hat = np.linalg.inv(index.T.dot(index)).dot(index.T).dot(data_flat)
    
    b0, b1 = beta_hat[0], beta_hat[1]
    b0 = b0.reshape((data.shape[1], data.shape[2]))
    b1 = b1.reshape((data.shape[1], data.shape[2]))

    return b0, b1